## Reviewing Charles' Work [here](https://github.com/Ficc-ai/ficc/blob/ficc_ml/ml_models/sequence_predictors/history_20230419.ipynb)



A few things to explore: 
1. Why is diff_ys the better label 
2. Paired t-test for model testing 
3. Custom callbacks

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import time
import gc

import numpy as np
from google.cloud import bigquery
from google.cloud import storage

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns
from pandas.tseries.offsets import BDay

from tensorflow.keras.layers import Embedding
from tensorflow.keras import activations
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from datetime import datetime
import matplotlib.pyplot as plt
import pickle5 as pickle


from ficc.utils.nelson_siegel_model import *
from ficc.utils.diff_in_days import *
from ficc.utils.auxiliary_functions import sqltodf


from IPython.display import display, HTML
import os


from ficc.data.process_data import process_data
from ficc.utils.auxiliary_variables import PREDICTORS, NON_CAT_FEATURES, BINARY, CATEGORICAL_FEATURES, IDENTIFIERS, PURPOSE_CLASS_DICT, NUM_OF_DAYS_IN_YEAR
from ficc.utils.gcp_storage_functions import upload_data, download_data
from ficc.utils.auxiliary_variables import RELATED_TRADE_BINARY_FEATURES, RELATED_TRADE_NON_CAT_FEATURES, RELATED_TRADE_CATEGORICAL_FEATURES

from ficc_keras_utils import *
import ficc_keras_utils

pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(f'TF Version: {tf.__version__}')

2023-05-11 16:32:35.640275: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-11 16:32:35.742960: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-11 16:32:35.744720: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Initializing pandarallel with 16.0 cores
INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
TF Version: 2.7.0


In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/jupyter/ficc/isaac_creds.json"
os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
pd.options.mode.chained_assignment = None

bq_client = bigquery.Client()
storage_client = storage.Client()

##COMMON VARIABLES
#DATA WINDOW
train_start = ficc_keras_utils.train_start
train_end = ficc_keras_utils.train_end
test_start = ficc_keras_utils.test_start
test_end = ficc_keras_utils.test_end
#MODEL PARAMETERS 
VALIDATION_SPLIT = ficc_keras_utils.VALIDATION_SPLIT
LEARNING_RATE = ficc_keras_utils.LEARNING_RATE
BATCH_SIZE = ficc_keras_utils.BATCH_SIZE
NUM_EPOCHS = ficc_keras_utils.NUM_EPOCHS
DROPOUT = ficc_keras_utils.DROPOUT

##NOTEBOOK SPECIFIC VARIABLES 
TRADE_SEQUENCE_LENGTH = 5
YIELD_SEQUENCE_LENGTH = 12
NUM_FEATURES = 6
# target_variable = 'new_ys_diff'
target_variable = 'new_ys'

Load data

In [3]:
from ficc_keras_utils import *

In [5]:
%%time

path = 'processed_file_FULL_2023-05-03-17:08_extended.pkl' #'processed_file_FULL_2023-05-03-17:08.pkl'#'../processed_file_FULL_2023-04-12-20:44.pkl'
data = load_data_from_pickle(path)

File not available, downloading from cloud storage and saving to processed_file_FULL_2023-05-03-17:08_extended.pkl
isaac_data/processed_file_FULL_2023-05-03-17:08_extended.pkl
CPU times: user 12min 27s, sys: 58.6 s, total: 13min 25s
Wall time: 24min 53s


In [6]:
data['new_ys'] = data['yield'] - data['new_ficc_ycl']
data['new_ys_realtime'] = data['yield'] - data['new_real_time_ficc_ycl']

In [7]:
data.trade_date.max(), data.trade_date.min()

(Timestamp('2023-04-28 00:00:00'), Timestamp('2022-09-01 00:00:00'))

# Testing Ideas

In [8]:
auxiliary_features = ['dollar_price',
                      'last_calc_date',
                     'calc_date', 
                     'trade_date',
                      'last_trade_date',
                     'trade_datetime', 
                     'purpose_sub_class', 
                     'called_redemption_type', 
                     'calc_day_cat',
                     'yield',
                     'ficc_ycl',
                     #'same_ys',
                     #'trade_history_sum',
                     'new_ficc_ycl',
                      'new_real_time_ficc_ycl',
                     'days_to_refund',
                      'last_dollar_price',
                      'last_rtrs_control_number',
                     'is_called',
                     ]

In [9]:
if 'target_attention_features' not in PREDICTORS:
    PREDICTORS.append('target_attention_features')
    
if 'ficc_treasury_spread' not in PREDICTORS:
    PREDICTORS.append('ficc_treasury_spread')
    NON_CAT_FEATURES.append('ficc_treasury_spread')
    
for col in ['new_ficc_ycl', 'new_real_time_ficc_ycl']:     
    if col not in PREDICTORS:
        PREDICTORS.append(col)
        NON_CAT_FEATURES.append(col)

# for col in ['extraordinary_make_whole_call', 'make_whole_call', 'has_unexpired_lines_of_credit']:     
#     if col not in data.columns:
#         try: 
#             print(f'Removing {col} from PREDICTORS and BINARY')
#             BINARY.remove(col)
#             PREDICTORS.remove(col) 
#         except:
#             continue

In [10]:
def process_data(data): 
    data['ted-rate'] = (data['t_rate_10'] - data['t_rate_2']) * 100
    
    # Here is a list of exclusions that we will be experimenting with. The model is trained with these exclusions. These exclusions were discussed with a team member.
    # Callable less than a year in the future
    # Maturity less than a year in the future and more than 30 years in the future
    
    data = data[(data.days_to_call == 0) | (data.days_to_call > np.log10(400))]
    data = data[(data.days_to_refund == 0) | (data.days_to_refund > np.log10(400))]
    data = data[(data.days_to_maturity == 0) | (data.days_to_maturity > np.log10(400))]
    data = data[data.days_to_maturity < np.log10(30000)]
    data['trade_history_sum'] = data.trade_history.parallel_apply(lambda x: np.sum(x))
    data.issue_amount = data.issue_amount.replace([np.inf, -np.inf], np.nan)
    data.dropna(inplace=True, subset=PREDICTORS+['trade_history_sum'])
    data.purpose_sub_class.fillna(0, inplace=True)
    
    # data['calc_date_duration'] = data[['last_calc_date','last_trade_date']].parallel_apply(get_calc_date_duration, axis=1)
    # data['new_ficc_ycl_fixed_shape'] = data[['trade_date', 'calc_date_duration']].parallel_apply(lambda x: calculate_ycl(x, new_yc_params), axis = 1)
    # data['new_ficc_ycl_prev_day'] = data[['last_calc_date', 'last_trade_date' ,'calc_date_duration','trade_date']].parallel_apply(get_yield_for_last_duration, axis=1)
    
    return data

In [11]:
%%time

processed_data = process_data(data) 
# processed_data = processed_data[IDENTIFIERS + PREDICTORS + auxiliary_features]

CPU times: user 59.4 s, sys: 26.4 s, total: 1min 25s
Wall time: 1min 30s


In [12]:
processed_data = processed_data.drop(51570)

In [13]:
processed_data.reset_index(inplace=True, drop=True)

In [14]:
encoders = {}
fmax = {}
for f in CATEGORICAL_FEATURES:
    print(f)
    fprep = preprocessing.LabelEncoder().fit(processed_data[f].drop_duplicates()) #note that there are apparently no trades with CC 
    fmax[f] = np.max(fprep.transform(fprep.classes_))
    encoders[f] = fprep
    
with open('encoders.pkl','wb') as file:
    pickle.dump(encoders,file)

rating
incorporated_state_code
trade_type
purpose_class


In [15]:
# %%time

# processed_data['yield_curve_history_sq'] = processed_data['yield_curve_history'].parallel_apply(lambda x: np.stack((x, np.square(x))))

In [16]:
processed_data.filter(regex='yield_curve').columns

Index(['yield_curve_history_5min_12', 'yield_curve_history_1min_20',
       'yield_curve_history_30min_5', 'yield_curve_history_30min_5_averaged',
       'yield_curve_history_1min_20_averaged',
       'yield_curve_history_5min_12_averaged'],
      dtype='object')

In [17]:
%%time

processed_data['yield_curve_history_5min_12_averaged_fixed'] = processed_data['yield_curve_history_5min_12_averaged'].apply(lambda x: x[::-1])

CPU times: user 2.57 s, sys: 123 ms, total: 2.69 s
Wall time: 2.69 s


In [18]:
%%time

processed_data['trade_history_fixed'] = processed_data['trade_history'].apply(lambda x: x[::-1, :])

CPU times: user 3.51 s, sys: 146 ms, total: 3.66 s
Wall time: 3.65 s


In [19]:
train_filter = (processed_data.trade_date < train_end) & (processed_data.trade_date >= train_start)
test_filter = (processed_data.trade_date >= test_start) & (processed_data.trade_date <test_end)
                                                            
train_dataframe = processed_data[train_filter]\
.sort_values(by='trade_date', ascending=True)\
.reset_index(drop=True)

test_dataframe = processed_data[test_filter]\
.sort_values(by='trade_date', ascending=True)\
.reset_index(drop=True)

print('Training data start: {}, end: {}'.format(train_dataframe.trade_date.min(),train_dataframe.trade_date.max()))
print('Test data start: {}, end: {}'.format(test_dataframe.trade_date.min(),test_dataframe.trade_date.max()))

Training data start: 2022-11-01 00:00:00, end: 2023-02-28 00:00:00
Test data start: 2023-03-01 00:00:00, end: 2023-03-31 00:00:00


In [22]:
def create_input(df):
    global encoders
    datalist = []
    # datalist.append(np.stack(df['yield_curve_history_sq'].to_numpy()))
    # datalist.append(np.stack(df['yield_curve_history'].to_numpy()))
    # datalist.append(np.stack(df['yield_curve_history_fixed'].to_numpy()))
    datalist.append(np.stack(df['yield_curve_history_5min_12_averaged_fixed'].to_numpy()))
    # datalist.append(np.stack(df['trade_history'].to_numpy()))
    datalist.append(np.stack(df['trade_history_fixed'].to_numpy()))
    datalist.append(np.stack(df['target_attention_features'].to_numpy()))

    noncat_and_binary = []
    for f in NON_CAT_FEATURES + BINARY:
        noncat_and_binary.append(np.expand_dims(df[f].to_numpy().astype('float32'), axis=1))
    datalist.append(np.concatenate(noncat_and_binary, axis=-1))
    
    for f in CATEGORICAL_FEATURES:
        encoded = encoders[f].transform(df[f])
        datalist.append(encoded.astype('float32'))
    
    return datalist

In [23]:
%%time
np.random.seed(0)
val_idx = np.random.choice(range(len(train_dataframe)), 
                 size = int(VALIDATION_SPLIT*len(train_dataframe)),
                 replace=False)

print(f'TRAINING DATA: N = {len(train_dataframe)-len(val_idx)}, MIN DATE = {train_dataframe.drop(val_idx, axis=0).trade_date.min()}, MAX DATE = {train_dataframe.drop(val_idx, axis=0).trade_date.max()}')
print(f'VALIDATION DATA: N = {len(val_idx)}, MIN DATE = {train_dataframe.iloc[val_idx].trade_date.min()}, MAX DATE = {train_dataframe.iloc[val_idx].trade_date.max()}')
print(f'TEST DATA: N = {len(test_dataframe)}, MIN DATE = {test_dataframe.trade_date.min()}, MAX DATE = {test_dataframe.trade_date.max()}')

x_train = create_input(train_dataframe.drop(val_idx, axis=0))
y_train = train_dataframe.drop(val_idx, axis=0)[target_variable]

x_val = create_input(train_dataframe.iloc[val_idx])
y_val = train_dataframe.iloc[val_idx][target_variable]

x_test = create_input(test_dataframe)
y_test = test_dataframe[target_variable]

TRAINING DATA: N = 2600040, MIN DATE = 2022-11-01 00:00:00, MAX DATE = 2023-02-28 00:00:00
VALIDATION DATA: N = 650010, MIN DATE = 2022-11-01 00:00:00, MAX DATE = 2023-02-28 00:00:00
TEST DATA: N = 708432, MIN DATE = 2023-03-01 00:00:00, MAX DATE = 2023-03-31 00:00:00
CPU times: user 40.7 s, sys: 2.55 s, total: 43.2 s
Wall time: 43.2 s


## Model Training and Testing

In [24]:
# Normalization layer for the yield history
yield_history_normalizer = Normalization(name='Yield_history_normalizer')
yield_history_normalizer.adapt(x_train[0],batch_size=BATCH_SIZE)

# Normalization layer for the trade history
trade_history_normalizer = Normalization(name='Trade_history_normalizer')
trade_history_normalizer.adapt(x_train[1],batch_size=BATCH_SIZE)

# Normalization layer for the non-categorical and binary features
noncat_binary_normalizer = Normalization(name='Numerical_binary_normalizer')
noncat_binary_normalizer.adapt(x_train[3], batch_size = BATCH_SIZE)

tf.keras.utils.set_random_seed(10)

2023-05-11 17:22:57.898258: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 17:22:57.902560: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-11 17:22:57.906173: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-11 17:22:57.908866: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [159]:
from keras.layers import LeakyReLU

def generate_model(TRADE_SEQUENCE_LENGTH = 5, YIELD_SEQUENCE_LENGTH = 12, NUM_FEATURES = NUM_FEATURES, trade_history_normalizer = trade_history_normalizer):
    inputs = []
    layer = []

    ############## INPUT BLOCK ###################
    yield_history_input = layers.Input(name="yield_curve_history_input", 
                                       # shape=(2, YIELD_SEQUENCE_LENGTH),
                                       shape=(YIELD_SEQUENCE_LENGTH, 1), 
                                       dtype = tf.float32) 
    
    trade_history_input = layers.Input(name="trade_history_input", 
                                       shape=(TRADE_SEQUENCE_LENGTH,NUM_FEATURES), 
                                       dtype = tf.float32) 

    target_attention_input = layers.Input(name="target_attention_input", 
                                       shape=(SEQUENCE_LENGTH, 3), 
                                       dtype = tf.float32) 

    inputs.append(yield_history_input)
    inputs.append(trade_history_input)
    inputs.append(target_attention_input)

    inputs.append(layers.Input(
        name="NON_CAT_AND_BINARY_FEATURES",
        shape=(len(NON_CAT_FEATURES + BINARY),)
    ))


    layer.append(noncat_binary_normalizer(inputs[3]))
    ####################################################


    ############## TRADE HISTORY MODEL #################

    lstm_layer = layers.LSTM(50, 
                             activation='tanh',
                             input_shape=(SEQUENCE_LENGTH,NUM_FEATURES),
                             return_sequences = True,
                             name='TRADE_HISTORY_LSTM')

    lstm_attention_layer = CustomAttention(50)

    lstm_layer_2 = layers.LSTM(100, 
                               activation='tanh',
                               input_shape=(SEQUENCE_LENGTH,50),
                               return_sequences = False,
                               name='TRADE_HISTORY_LSTM_2')


    features = lstm_layer(trade_history_normalizer(inputs[1]))
    features = lstm_attention_layer(features, features, inputs[2])
    features = layers.BatchNormalization()(features)
    # features = layers.Dropout(DROPOUT)(features)

    features = lstm_layer_2(features)
    features = layers.BatchNormalization()(features)
    # features = layers.Dropout(DROPOUT)(features)

    trade_history_output = layers.Dense(100, 
                                        activation='relu')(features)

    ####################################################
    
    ############## YIELD HISTORY MODEL #################

    yield_lstm_layer = layers.LSTM(50, 
                             activation='tanh',
                             input_shape=(1, SEQUENCE_LENGTH),
                             return_sequences = False,
                             name='Yield_History_LSTM')

   
    # constant = tf.keras.layers.Lambda(lambda x:x+tf.constant(np.random.rand(12).astype('float32')))(inputs[0]) 
    # yield_features = tf.keras.layers.Add()([inputs[0], constant])
    yield_features = yield_lstm_layer(yield_history_normalizer(inputs[0]))
    # yield_features = yield_lstm_layer(yield_features) #yield_lstm_layer(yield_history_normalizer(inputs[0]))
    yield_features = layers.Dense(25, activation='relu')(yield_features)
    # yield_history_output = layers.LeakyReLU(alpha = 0.05)(yield_features)

    yield_history_output =  layers.BatchNormalization()(yield_features)
    ####################################################

    ############## REFERENCE DATA MODEL ################
    global encoders
    for f in CATEGORICAL_FEATURES:
        fin = layers.Input(shape=(1,), name = f)
        inputs.append(fin)
        embedded = layers.Flatten(name = f + "_flat")( layers.Embedding(input_dim = fmax[f]+1,
                                                                        output_dim = max(30,int(np.sqrt(fmax[f]))),
                                                                        input_length= 1,
                                                                        name = f + "_embed")(fin))
        layer.append(embedded)


    reference_hidden = layers.Dense(400,
                                    activation='relu',
                                    name='reference_hidden_1')(layers.concatenate(layer, axis=-1))

    reference_hidden = layers.BatchNormalization()(reference_hidden)
    reference_hidden = layers.Dropout(DROPOUT)(reference_hidden)

    reference_hidden2 = layers.Dense(200,activation='relu',name='reference_hidden_2')(reference_hidden)
    reference_hidden2 = layers.BatchNormalization()(reference_hidden2)
    reference_hidden2 = layers.Dropout(DROPOUT)(reference_hidden2)

    reference_output = layers.Dense(100,activation='tanh',name='reference_hidden_3')(reference_hidden2)

    ####################################################

    feed_forward_input = layers.concatenate([yield_history_output, reference_output, trade_history_output])

    hidden = layers.Dense(300,activation='relu')(feed_forward_input)
    hidden = layers.BatchNormalization()(hidden)
    hidden = layers.Dropout(DROPOUT)(hidden)

    hidden2 = layers.Dense(100,activation='tanh')(hidden)
    hidden2 = layers.BatchNormalization()(hidden2)
    hidden2 = layers.Dropout(DROPOUT)(hidden2)
    
#     hidden = layers.Dense(300, activation=)(feed_forward_input)
#     hidden = layers.LeakyReLU(alpha=0.05)(hidden)
#     hidden = layers.BatchNormalization()(hidden)
#     hidden = layers.Dropout(DROPOUT)(hidden)

#     hidden2 = layers.Dense(100)(hidden)
#     hidden2 = layers.LeakyReLU(alpha=0.05)(hidden2)
#     hidden2 = layers.BatchNormalization()(hidden2)
#     hidden2 = layers.Dropout(DROPOUT)(hidden2)

    final = layers.Dense(1)(hidden2)

    model = keras.Model(inputs=inputs, outputs=final)
    
    return model

def create_tf_data(x_train, y_train, shuffle=False, shuffle_buffer=1):
                     
    X=()
    for x in x_train:
        X += (tf.data.Dataset.from_tensor_slices(x),)
        

    temp = tf.data.Dataset.zip((X))
    del X
    dataset = tf.data.Dataset.zip((temp,
                        tf.data.Dataset.from_tensor_slices(y_train)))
    del temp
    if shuffle:
        shuffle_buffer = int(len(x_train[0])*shuffle_buffer)
        dataset = dataset.shuffle(shuffle_buffer)
        
    return dataset

In [160]:
def train_model(x_train, y_train, x_val, y_val, shuffle, shuffle_buffer=1):
    tf.keras.backend.clear_session()
    gc.collect()

    timestamp = datetime.now().strftime('%Y-%m-%d %H-%M')
    
    fit_callbacks = fit_callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        verbose=0,
        mode="auto",
        restore_best_weights=True),
        # time_callback,
        CSVLoggerTimeHistory(timestamp+'_training_logs_yield_history.csv', separator=",", append=False)]
    
    with tf.device('/cpu:0'):
        train_ds = create_tf_data(x_train, y_train, shuffle, shuffle_buffer)
        train_ds = train_ds.batch(BATCH_SIZE).prefetch(2).cache()
        val_ds = create_tf_data(x_val, y_val, shuffle = False)
        val_ds = val_ds.batch(BATCH_SIZE).prefetch(2).cache()

    model_new_ys = generate_model(TRADE_SEQUENCE_LENGTH=TRADE_SEQUENCE_LENGTH,
                                  YIELD_SEQUENCE_LENGTH=YIELD_SEQUENCE_LENGTH,
                                  NUM_FEATURES=6, 
                                  trade_history_normalizer = trade_history_normalizer)
    
    model_new_ys.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
          loss=keras.losses.MeanAbsoluteError(),
          metrics=[keras.metrics.MeanAbsoluteError()])

    history_new_ys = model_new_ys.fit(train_ds,
                                      validation_data=val_ds,
                                        epochs=NUM_EPOCHS,     
                                        verbose=1, 
                                        callbacks=fit_callbacks,
                                        use_multiprocessing=True,
                                        workers=8)
    
    return history_new_ys, model_new_ys

In [ ]:
results = []
predictions = []

for i in range(8):
    history, model = train_model(x_train, y_train, x_val, y_val, shuffle=True, shuffle_buffer=0.75)
    pred = model.predict(x_test)
    predictions.append(pred)
    print('='*25+f' TRIAL {i}, MAE: {mean_absolute_error(pred,y_test)} '+'='*25)
    results.append([history, model])

Epoch 1/100


2023-05-10 21:50:54.708102: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 859703 of 1950030
2023-05-10 21:51:04.708107: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 1606317 of 1950030
2023-05-10 21:51:10.619188: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:405] Shuffle buffer filled.


2540/2540 [==============================] - 102s 27ms/step - loss: 17.0318 - mean_absolute_error: 17.0318 - val_loss: 10.7192 - val_mean_absolute_error: 10.7192
Epoch 2/100
2540/2540 [==============================] - 50s 20ms/step - loss: 11.0121 - mean_absolute_error: 11.0121 - val_loss: 10.4007 - val_mean_absolute_error: 10.4007
Epoch 3/100
2540/2540 [==============================] - 46s 18ms/step - loss: 10.7433 - mean_absolute_error: 10.7433 - val_loss: 10.5005 - val_mean_absolute_error: 10.5005
Epoch 4/100
2540/2540 [==============================] - 51s 20ms/step - loss: 10.5510 - mean_absolute_error: 10.5510 - val_loss: 9.8806 - val_mean_absolute_error: 9.8806
Epoch 5/100
2540/2540 [==============================] - 50s 20ms/step - loss: 10.4192 - mean_absolute_error: 10.4192 - val_loss: 9.9585 - val_mean_absolute_error: 9.9585
Epoch 6/100
2540/2540 [==============================] - 51s 20ms/step - loss: 10.3235 - mean_absolute_error: 10.3235 - val_loss: 9.6601 - val_mean_ab

2023-05-10 22:31:04.613791: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 754782 of 1950030
2023-05-10 22:31:14.613747: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 1454351 of 1950030


   1/2540 [..............................] - ETA: 22:10:02 - loss: 56.7734 - mean_absolute_error: 56.7734

2023-05-10 22:31:20.230682: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:405] Shuffle buffer filled.


2540/2540 [==============================] - 102s 28ms/step - loss: 16.9290 - mean_absolute_error: 16.9290 - val_loss: 10.6959 - val_mean_absolute_error: 10.6959
Epoch 2/100
2540/2540 [==============================] - 49s 19ms/step - loss: 11.0208 - mean_absolute_error: 11.0208 - val_loss: 10.4035 - val_mean_absolute_error: 10.4035
Epoch 3/100
2540/2540 [==============================] - 50s 20ms/step - loss: 10.7540 - mean_absolute_error: 10.7540 - val_loss: 10.1139 - val_mean_absolute_error: 10.1139
Epoch 4/100
2540/2540 [==============================] - 44s 17ms/step - loss: 10.5633 - mean_absolute_error: 10.5633 - val_loss: 10.0646 - val_mean_absolute_error: 10.0646
Epoch 5/100
2540/2540 [==============================] - 49s 19ms/step - loss: 10.4260 - mean_absolute_error: 10.4260 - val_loss: 9.7275 - val_mean_absolute_error: 9.7275
Epoch 6/100
2540/2540 [==============================] - 50s 20ms/step - loss: 10.3037 - mean_absolute_error: 10.3037 - val_loss: 9.8584 - val_mean_

2023-05-10 23:34:45.900586: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 822141 of 1950030
2023-05-10 23:34:55.900567: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:380] Filling up shuffle buffer (this may take a while): 1692162 of 1950030
2023-05-10 23:34:58.841606: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:405] Shuffle buffer filled.


2540/2540 [==============================] - 103s 29ms/step - loss: 16.9041 - mean_absolute_error: 16.9041 - val_loss: 10.5899 - val_mean_absolute_error: 10.5899
Epoch 2/100
2540/2540 [==============================] - 41s 16ms/step - loss: 11.0437 - mean_absolute_error: 11.0437 - val_loss: 10.2778 - val_mean_absolute_error: 10.2778
Epoch 3/100
2540/2540 [==============================] - 49s 19ms/step - loss: 10.7641 - mean_absolute_error: 10.7641 - val_loss: 10.1108 - val_mean_absolute_error: 10.1108
Epoch 4/100
2540/2540 [==============================] - 50s 20ms/step - loss: 10.5758 - mean_absolute_error: 10.5758 - val_loss: 10.0413 - val_mean_absolute_error: 10.0413
Epoch 5/100
2540/2540 [==============================] - 51s 20ms/step - loss: 10.4380 - mean_absolute_error: 10.4380 - val_loss: 9.9485 - val_mean_absolute_error: 9.9485
Epoch 6/100
2540/2540 [==============================] - 49s 19ms/step - loss: 10.3327 - mean_absolute_error: 10.3327 - val_loss: 9.9326 - val_mean_

In [167]:
for i, prediction in enumerate(predictions):
    print(f'{mean_absolute_error(prediction, y_test)}')

9.727299753801523
9.717669412464797
9.740593986327768
9.750344895786744
9.67288864488992
9.755444177230974
9.662624881636848
9.82704817137997


# Save Results

In [91]:
pred_df = pd.DataFrame(np.stack(predictions).reshape(8, -1).T, columns = [f'model_{i}' for i in range(1,9)])
test_dataframe[[f'prediction_{i}' for i in range(1,9)]] = pred_df

In [93]:
test_dataframe.to_pickle('diff_ys-yield_history_2-2023_05_09.pkl')